In [1]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import re
from sklearn.preprocessing import RobustScaler
import scipy.stats as ss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [7]:
os.listdir('/Users/kadima/experiment_any/anomaly-detection/datasets')

['lympho.mat',
 'wbc.mat',
 'breast-cancer-wisconsin.names',
 'breast-cancer-wisconsin.data']

In [8]:
with open("./Parkinson_withoutdupl_75.txt",'r') as f:
    data = f.readlines()
# print(data[28:-2])
col_names = []
list_ = []
for id_, line in enumerate(data):
    if line.startswith("@ATTRIBUTE"):
        colName = re.findall("@ATTRIBUTE '(.*?)'", line)[0]
        col_names.append(colName)
    if id_ >= 28 and id_<= len(data) - 1 and line != "\n":
        new_line = line.split(",")
        new_line = [x.strip() for x in new_line]
        new_line = [float(new_line[x]) for x in range(len(new_line)-1)] + [new_line[-1].replace("'","")]
        list_.append(new_line)
        
df = pd.DataFrame(list_, columns=col_names)
df['outlier'] = df.outlier.apply(lambda x: 1 if x =='yes' else 0)

In [9]:
df_sampled = df.sample(frac=1).reset_index(drop=True)
X = df_sampled.iloc[:,1:-1]
y_ground_truth = df_sampled.iloc[:,-1]

In [10]:
X

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
0,127.930,138.752,112.173,0.00605,0.00005,0.00321,0.00318,0.00962,0.03235,0.339,...,0.03105,0.04079,0.02663,19.651,0.654945,0.675865,-5.498456,0.234196,2.103014,0.216638
1,114.847,271.314,104.680,0.00867,0.00008,0.00373,0.00520,0.01120,0.03225,0.350,...,0.02519,0.05414,0.01143,21.660,0.547975,0.817396,-4.609161,0.221711,1.831691,0.316395
2,208.519,220.315,199.020,0.00609,0.00003,0.00368,0.00339,0.01105,0.01761,0.155,...,0.01307,0.02855,0.00830,22.407,0.338097,0.712466,-6.471427,0.184378,2.502336,0.136390
3,206.327,226.355,92.020,0.00495,0.00002,0.00302,0.00246,0.00905,0.02105,0.209,...,0.01604,0.03320,0.01048,21.864,0.331508,0.715121,-6.729713,0.181701,2.938114,0.115515
4,115.380,123.109,108.634,0.00332,0.00003,0.00160,0.00199,0.00480,0.01503,0.137,...,0.01133,0.02436,0.00401,26.005,0.405991,0.761255,-5.966779,0.197938,1.974857,0.184067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,210.141,232.706,185.258,0.00534,0.00003,0.00321,0.00280,0.00964,0.01680,0.149,...,0.01301,0.02583,0.00620,23.671,0.441097,0.722254,-5.963040,0.250283,2.489191,0.177807
191,148.090,162.824,67.343,0.00762,0.00005,0.00467,0.00354,0.01400,0.05428,0.497,...,0.03635,0.10070,0.02431,21.718,0.487407,0.727313,-6.261141,0.120956,2.137075,0.141958
192,198.116,233.099,174.478,0.02714,0.00014,0.01568,0.01699,0.04705,0.08684,1.018,...,0.08318,0.12851,0.16265,9.449,0.584164,0.754449,-2.839756,0.356224,3.184027,0.454721
193,176.824,215.293,83.961,0.00460,0.00003,0.00209,0.00221,0.00628,0.01169,0.117,...,0.01104,0.01603,0.01161,27.166,0.400088,0.656182,-4.711007,0.281618,2.655744,0.234809


- 孤立森林不用归一化
- LOF 要做归一化

In [11]:
ro_scaler = RobustScaler()
X_scaled_robust = ro_scaler.fit_transform(X)

# 打分矩阵

let's say we have 200 detectors

In [12]:
score_matrix = np.zeros([X.shape[0], 10])

# 孤立森林

In [13]:
isolate_clfs = []
for i in range(5):
    n_est = 100
    clf = IsolationForest(n_estimators=n_est,contamination=0.5)
    clf.fit(X)
    output_score = abs(clf.score_samples(X))
    score_matrix[:, i] = output_score
    isolate_clfs.append(clf)

# LOF with scaled data

In [14]:
lof_clfs = []
for i in range(5):
    n_nb = np.random.choice(range(5,20))
    clf = LocalOutlierFactor(n_neighbors=n_nb)
    clf.fit(X_scaled_robust)
    score_ = abs(clf.negative_outlier_factor_)
    score_matrix[:, i+5] = score_
    lof_clfs.append(clf)

In [15]:
score_matrix

array([[0.40108802, 0.38421255, 0.4010765 , ..., 1.00740911, 1.00917371,
        1.00587199],
       [0.46023578, 0.42690163, 0.44955457, ..., 1.07863342, 1.08900226,
        1.13605142],
       [0.43454773, 0.41442978, 0.45449004, ..., 1.24248553, 1.31727942,
        1.34297865],
       ...,
       [0.66603446, 0.67315376, 0.65043818, ..., 1.39385669, 1.42148479,
        1.42855563],
       [0.42968634, 0.42901625, 0.43617137, ..., 1.15318336, 1.20069552,
        1.14951026],
       [0.53039386, 0.53137492, 0.51962237, ..., 1.27846076, 1.17234784,
        1.11175718]])

In [16]:
score_matrix.shape

(195, 10)

# disagreement matrix

## 1. ranking disagreement

In [17]:
rank_matrix = np.zeros([len(X), 10])

In [18]:
for i in range(10):
    # rank by each column and get its rank_position
    rank_matrix[:,i] = ss.rankdata(score_matrix[:,i])
rank_matrix[:5]

array([[ 55.,  13.,  38.,  45.,  49.,  16.,  25.,  50.,  41.,  35.],
       [153., 124., 136., 134., 126., 120., 127., 112., 119., 148.],
       [122.,  92., 139.,  97., 102., 140., 149., 173., 188., 188.],
       [ 96.,  91., 101.,  90.,  58., 128., 131., 152., 118., 150.],
       [  4.,  10.,  13.,   3.,   3.,   3.,   5.,  12.,  25.,  49.]])

calculate standard deviation

In [19]:
std_record = np.zeros(len(X))
for i in range(195):
    std_record[i] = np.std(rank_matrix[i,:])

In [20]:
std_record

array([13.71896498, 12.29186723, 34.19064199, 28.12916636, 13.76263056,
       13.60735095, 35.04054794, 39.8282563 , 26.43179903, 42.61736735,
       14.47376938, 27.72525924, 30.01083138, 12.89340917,  8.56504524,
        8.57263087, 30.95561338, 42.78971839, 17.16857595,  9.2173749 ,
       11.35825691, 39.75235842, 52.20584258,  4.65295605, 20.88061302,
       21.66771792, 15.06519167, 40.98890094, 32.25972722, 29.52642884,
        9.60416576, 31.16793224, 48.14862407, 26.36380094,  0.5       ,
        8.60232527, 12.87788803, 36.74782306, 15.83161394,  2.54754784,
       48.9035786 , 24.7549995 , 12.92323489, 21.44970862, 22.47754435,
       12.06192356, 18.73632835, 19.04757202, 19.4257561 , 37.92571687,
       24.78225171, 22.41567532, 27.97159273, 32.50307678, 16.1752898 ,
       21.61388443,  6.48151217, 21.13030998, 27.88207309, 21.72095762,
       35.27052594, 48.6671347 , 31.57467339, 11.13058848, 21.14450283,
       24.1091269 ,  7.36545993, 39.0262732 , 18.53887807, 23.27

get top 5 and get the middle score

In [21]:
std_max_rows = np.argsort(std_record)[-5:]
std_max_rows

array([ 83, 153, 190,  76, 103])

In [22]:
print(rank_matrix[std_max_rows])
print()
print(std_record[std_max_rows])

[[ 18.  39.  21.  47.  45. 112.  81. 135. 152. 172.]
 [189. 186. 188. 185. 186. 174. 182. 103.  73.  34.]
 [ 44.  50.  92.  43.  66. 143. 138. 160. 183. 185.]
 [ 61.  31.  15.  70.  39. 119. 104. 153. 176. 170.]
 [186. 184. 184. 187. 187. 184. 187. 140.  48.  36.]]

[53.82341498 54.75034246 54.8        56.21885805 56.84901055]


so, we get the score threshold:

In [23]:
score_matrix[std_max_rows[-1]]

array([0.56572759, 0.54171094, 0.55097216, 0.56339741, 0.56935567,
       1.48989399, 1.5758991 , 1.13131512, 1.01729151, 1.00614312])

# comparison
## 1. SD
## 2. IQR
## 3. MAD
## 4. 2T
## 5. clever standard deviation
## 6. disagreement

----

lof_clfs, isolate_clfs

In [24]:
perform_records_base = {"SD":{}, "IQR":{}, "MAD":{}, "2T":{}, "Disagreement":{}}
for i in ["eucledian", "scaled_eucledian",]+ ["clf_"+str(j+1) for j in range(10)]:
    for key in perform_records_base.keys():
        perform_records_base[key][i]=0
acc_perform = perform_records_base.copy()
f1_perform = perform_records_base.copy()

### 1. SD_model

In [25]:
for i in range(score_matrix.shape[1]):
    high_limit = np.mean(score_matrix[:, i]) + 3*np.std(score_matrix[:, i])
    y_predict = score_matrix[:,i]>high_limit
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["SD"]["clf_"+str(i+1)] = f1
    acc_perform["SD"]["clf_"+str(i+1)] = acc

### IQR model


In [26]:
for i in range(score_matrix.shape[1]):
    iqr = np.percentile(score_matrix[:,i],75) - np.percentile(score_matrix[:,i],25)
    y_predict = score_matrix[:,i]> np.percentile(score_matrix[:,i],75)+1.5*iqr
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["IQR"]["clf_"+str(i+1)] = f1
    acc_perform["IQR"]["clf_"+str(i+1)] = acc
    

### MAD model

In [27]:
for i in range(score_matrix.shape[1]):
    median_ = np.median(score_matrix[:,i])
    mad = 1.4826*np.median(np.abs(score_matrix[:,i]-median_))
    y_predict = score_matrix[:,i]> 3*mad
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["MAD"]["clf_"+str(i+1)] = f1
    acc_perform["MAD"]["clf_"+str(i+1)] = acc

### disagreement

In [28]:
for i in range(score_matrix.shape[1]):
    y_predict = score_matrix[:,i] > score_matrix[std_max_rows[-1]][i]
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["Disagreement"]["clf_"+str(i+1)] = f1
    acc_perform["Disagreement"]["clf_"+str(i+1)] = acc

In [29]:
f1_perform

{'SD': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.2717948717948718,
  'clf_2': 0.27692307692307694,
  'clf_3': 0.2717948717948718,
  'clf_4': 0.2717948717948718,
  'clf_5': 0.26666666666666666,
  'clf_6': 0.26153846153846155,
  'clf_7': 0.26666666666666666,
  'clf_8': 0.2564102564102564,
  'clf_9': 0.2564102564102564,
  'clf_10': 0.2564102564102564},
 'IQR': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.3128205128205128,
  'clf_2': 0.3230769230769231,
  'clf_3': 0.3128205128205128,
  'clf_4': 0.3128205128205128,
  'clf_5': 0.3128205128205128,
  'clf_6': 0.2923076923076923,
  'clf_7': 0.3230769230769231,
  'clf_8': 0.27692307692307694,
  'clf_9': 0.28205128205128205,
  'clf_10': 0.2717948717948718},
 'MAD': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.7538461538461538,
  'clf_2': 0.7538461538461538,
  'clf_3': 0.7538461538461538,
  'clf_4': 0.7538461538461538,
  'clf_5': 0.7538461538461538,
  'clf_6': 0.7538461538461538,
  'clf_7': 0.7538461538461538,
  'c

## 2T-SD

In [30]:
for i in range(score_matrix.shape[1]):
    high_limit = np.mean(score_matrix[:, i]) + 3*np.std(score_matrix[:, i])
    std2 = np.std(score_matrix[:,i]<=high_limit)
    high_limit2 = np.mean(score_matrix[score_matrix[:,i]<=high_limit, i]) + 3*std2
    y_predict = score_matrix[:,i]> high_limit2
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["2T"]["clf_"+str(i+1)] = f1
    acc_perform["2T"]["clf_"+str(i+1)] = acc

In [31]:
f1_perform

{'SD': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.2717948717948718,
  'clf_2': 0.27692307692307694,
  'clf_3': 0.2717948717948718,
  'clf_4': 0.2717948717948718,
  'clf_5': 0.26666666666666666,
  'clf_6': 0.26153846153846155,
  'clf_7': 0.26666666666666666,
  'clf_8': 0.2564102564102564,
  'clf_9': 0.2564102564102564,
  'clf_10': 0.2564102564102564},
 'IQR': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.3128205128205128,
  'clf_2': 0.3230769230769231,
  'clf_3': 0.3128205128205128,
  'clf_4': 0.3128205128205128,
  'clf_5': 0.3128205128205128,
  'clf_6': 0.2923076923076923,
  'clf_7': 0.3230769230769231,
  'clf_8': 0.27692307692307694,
  'clf_9': 0.28205128205128205,
  'clf_10': 0.2717948717948718},
 'MAD': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.7538461538461538,
  'clf_2': 0.7538461538461538,
  'clf_3': 0.7538461538461538,
  'clf_4': 0.7538461538461538,
  'clf_5': 0.7538461538461538,
  'clf_6': 0.7538461538461538,
  'clf_7': 0.7538461538461538,
  'c